In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_pandas = pd.read_csv("../data/midcp_wave.csv")

INDEX = 'MIDCP'
# INDEX = 'BNF'
# INDEX = 'NIFTY'
# INDEX = 'FNF'
# INDEX = 'STOCK'
# INDEX = 'SENSEX'

if INDEX == 'BNF':
    lev_ = 5.066
    slippage_ = 0.0001
    lot_size_ = 15
elif INDEX == 'NIFTY' or INDEX == 'FNF':
    lev_ = 6.333
    slippage_ = 0.0002
    lot_size_ = 25
elif INDEX == 'MIDCP':
    lev_ = 6.333
    slippage_ = 0.0005
    lot_size_ = 50
elif INDEX == 'SENSEX':
    lev_ = 7
    slippage_ = 0.0001
    lot_size_ = 10
elif INDEX == 'STOCK':
    lev_ = 5
    slippage_ = 0.001
    lot_size_ = 1
# bnf_pandas = pd.read_csv("../data/finnifty_1hr_tv (2).csv")
# bnf_pandas = pd.read_csv('../data/midcp_1hr_tv (4).csv')
# bnf_pandas = pd.read_csv('../data/finnifty_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/bnf_fut_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')


In [6]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2015]
bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [7]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

<class 'polars.dataframe.frame.DataFrame'>


In [8]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


In [9]:
# Parameters
ema_short_period = 9
ema_long_period = 21
rsi_period = 14
rsi_short_threshold = 50
rsi_oversold = 30
# stop_loss_pct = 0.5 / 100  # 0.5%
# take_profit_pct = 1.0 / 100  # 1.0%

In [10]:
df = bnf_pandas

In [11]:
# df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
# df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

In [12]:
# Calculate RSI
# delta = df["close"].diff()
# gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
# loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
# rs = gain / loss
# df["rsi"] = 100 - (100 / (1 + rs))

In [26]:
def short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold):

    df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
    df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

    # Calculate RSI
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
    rs = gain / loss
    df["rsi"] = 100 - (100 / (1 + rs))
    
    # Initialize variables
    position = None  # None, 'short'
    tradebook = []
    signal_time = None
    signal_price = None
    entry_time = None
    entry_price = None
    initial_sl = None
    
    # Iterate through rows to generate signals
    for i in range(len(df)):
        row = df.iloc[i]
    
        # Check for short signal condition
        if position is None:
            if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                if signal_price is None:  # New signal
                    signal_time = row["datetime"]
                    signal_price = row["low"]
                else:
                    if row['low'] >= signal_price:
                        # Update Signal Price to Better Candle low
                        signal_time = row["datetime"]
                        signal_price = row["low"]
    
            if signal_price is not None:
                if row["low"] < signal_price and row['high'] >= signal_price:  # Signal broken
                    entry_time = row["datetime"]
                    entry_price = signal_price
                    initial_sl = max(df["high"][max(0, i - 3):i].max(), row["high"])
                    position = "short"
        
                    # Add trade to tradebook
                    tradebook.append({
                        "Signal Time": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": None,
                        "Exit Price": None,
                        "Points Captured": None
                    })
                    signal_time, signal_price = None, None  # Reset signal
                else:
                    signal_price = None
                    if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                        if signal_price is None:  # New signal
                            signal_time = row["datetime"]
                            signal_price = row["low"]
                        else:
                            if row['low'] >= signal_price:
                                # Update Signal Price to Better Candle low
                                signal_time = row["datetime"]
                                signal_price = row["low"]
    
        # Exit logic for short position
        if position is not None:
            if row['close'] >= initial_sl:
                exit_time = row["datetime"]
                exit_price = row['close']
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'Initial SL Hit'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
            elif row["rsi"] > rsi_oversold:
                exit_time = row["datetime"]
                exit_price = row["close"]
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'RSI Oversold'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
    
    # Convert tradebook to a DataFrame
    tradebook_df = pd.DataFrame(tradebook)
    return tradebook_df
    # Output
    # print(tradebook_df)
    
    # Optional: Save to CSV
    # tradebook_df.to_csv("tradebook.csv", index=False)


In [38]:
ema_short_period = 7
ema_long_period = 23
rsi_period = 4
rsi_short_threshold = 70
rsi_oversold = 20
tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
# tradebook_df

In [39]:
tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
tradebook_df['PnL w cs'] = (tradebook_df['Points Captured'] - slippage_*(tradebook_df['Entry Price'] + tradebook_df['Exit Price'])) * tradebook_df['Qty']
tradebook_df['ROI%'] = tradebook_df['PnL w cs'] * 100 / 10000000

In [40]:
tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year

In [41]:
tradebook_df['ROI%'].sum()

56.23348722862261

In [42]:
tradebook_df[tradebook_df['Trade Year'] >= 2024]

,Signal Time,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Points Captured,Remark,Qty,PnL,PnL w cs,ROI%,Trade Year
227,2024-01-09 09:15:00,2024-01-09 10:15:00,10481.5000,10587.4000,2024-01-09 10:15:00,10509.2000,-27.7000,RSI Oversold,4770.3096,-132137.5757,-182203.6445,-1.8220,2024
228,2024-01-09 13:15:00,2024-01-09 14:15:00,10455.9000,10512.0500,2024-01-10 13:15:00,10355.5000,100.4000,RSI Oversold,4781.9891,480111.7073,430351.7631,4.3035,2024
229,2024-01-17 10:15:00,2024-01-17 11:15:00,10478.9000,10561.1500,2024-01-17 12:15:00,10475.2500,3.6500,RSI Oversold,4771.4932,17415.9501,-32575.3419,-0.3258,2024
230,2024-01-17 13:15:00,2024-01-17 14:15:00,10464.8000,10525.0000,2024-01-17 14:15:00,10445.9500,18.8500,RSI Oversold,4777.9222,90063.8330,40108.8650,0.4011,2024
231,2024-01-17 15:15:00,2024-01-18 09:15:00,10417.6000,10508.9000,2024-01-18 10:15:00,10422.3500,-4.7500,RSI Oversold,4799.5700,-22797.9573,-72809.3563,-0.7281,2024
232,2024-01-23 12:15:00,2024-01-23 13:15:00,10429.0500,10590.1000,2024-01-24 09:15:00,10452.3000,-23.2500,RSI Oversold,4794.3005,-111467.4875,-161523.2212,-1.6152,2024
233,2024-01-24 10:15:00,2024-01-24 11:15:00,10444.4000,10495.3500,2024-01-24 11:15:00,10380.2000,64.2000,RSI Oversold,4787.2544,307341.7334,257495.4042,2.5750,2024
234,2024-01-25 11:15:00,2024-01-25 12:15:00,10426.0500,10606.2500,2024-01-25 14:15:00,10416.7000,9.3500,RSI Oversold,4795.6801,44839.6085,-5137.9717,-0.0514,2024
235,2024-02-09 12:15:00,2024-02-09 13:15:00,10667.8000,10733.3000,2024-02-09 13:15:00,10684.9000,-17.1000,RSI Oversold,4687.0020,-80147.7343,-130187.8082,-1.3019,2024
236,2024-02-12 09:15:00,2024-02-12 10:15:00,10711.7000,10812.6500,2024-02-12 10:15:00,10724.2500,-12.5500,RSI Oversold,4667.7932,-58580.8042,-108610.0946,-1.0861,2024


In [43]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2022, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2022, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [44]:
variation = f'...{rsi_oversold}'
stats = generate_stats(tradebook_df, variation)

In [45]:
stats_dictionary = {}
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2022      72.0452          134  38.8060                3.8006             -1.5315     -23.5743       3.0561     ...20
2023     -29.9678           93  31.1828                2.0236             -1.3852     -36.6251      -0.8182     ...20
2024      14.1560           98  32.6531                4.0516             -1.7499     -25.8244       0.5482     ...20
Overall   56.2335          325  34.7692                3.4156             -1.5553     -50.1203       1.1220     ...20


In [36]:
stats_dictionary = {}

for ema_short_period in range(5, 16, 2):
    for ema_long_period in range(5, 24, 2):
        for rsi_period in range(2, 15, 2):
            for rsi_short_threshold in range(20, 81, 10):
                for rsi_oversold in range(20, 61, 10):
                    if (ema_short_period < ema_long_period) and (rsi_oversold <= rsi_short_threshold):
                        tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
                        tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
                        tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
                        tradebook_df['PnL w cs'] = (tradebook_df['Points Captured'] - slippage_*(tradebook_df['Entry Price'] + tradebook_df['Exit Price'])) * tradebook_df['Qty']
                        tradebook_df['ROI%'] = tradebook_df['PnL w cs'] * 100 / 10000000
                        tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year
                        variation = f'{ema_short_period}, {ema_long_period}, {rsi_period}, {rsi_short_threshold}, {rsi_oversold} -> {round(tradebook_df["ROI%"].sum())}%'
                        print(variation)
                        new_tb = tradebook_df
                        if len(new_tb) > 1:
                            new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                            stats = generate_stats(new_tb, variation)
                            for overall_roi_dd_ratio, stats_df in stats.items():
                                if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 9:
                                    print(stats_df.to_string())
                                    stats_dictionary[overall_roi_dd_ratio] = stats_df

5, 7, 2, 20, 20 -> 167%
5, 7, 2, 30, 20 -> 169%
5, 7, 2, 30, 30 -> 158%
5, 7, 2, 40, 20 -> 156%
5, 7, 2, 40, 30 -> 143%
5, 7, 2, 40, 40 -> 123%
5, 7, 2, 50, 20 -> 165%
5, 7, 2, 50, 30 -> 152%
5, 7, 2, 50, 40 -> 125%
5, 7, 2, 50, 50 -> 112%
5, 7, 2, 60, 20 -> 173%
5, 7, 2, 60, 30 -> 166%
5, 7, 2, 60, 40 -> 138%
5, 7, 2, 60, 50 -> 112%
5, 7, 2, 60, 60 -> 122%
5, 7, 2, 70, 20 -> 195%
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                Variation
2022     139.3621          127  49.6063                4.2208             -1.9773     -20.1246       6.9250  5, 7, 2, 70, 20 -> 195%
2023      13.1231          107  40.1869                2.2351             -1.2966     -14.3391       0.9152  5, 7, 2, 70, 20 -> 195%
2024      42.9825          115  45.2174                2.9215             -1.7291     -19.2190       2.2365  5, 7, 2, 70, 20 -> 195%
Overall  195.4677          349  45.2722                3.2528             -1.6674   

In [37]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

{13.461489916909056:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2022     138.9757          134  55.2239                3.0951   
 2023      16.5240           93  44.0860                1.8467   
 2024      63.1245           98  43.8776                3.4500   
 Overall  218.6242          325  48.6154                2.8677   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio  \
 2022                -1.5010     -15.0668       9.2240   
 2023                -1.1383     -16.2407       1.0174   
 2024                -1.5496     -14.5690       4.3328   
 Overall             -1.4041     -16.2407      13.4615   
 
                         Variation  
 2022     7, 23, 4, 70, 20 -> 219%  
 2023     7, 23, 4, 70, 20 -> 219%  
 2024     7, 23, 4, 70, 20 -> 219%  
 Overall  7, 23, 4, 70, 20 -> 219%  ,
 13.06686696195676:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2022     162.6417          118  55.0847                3.7535   
 2023      18.8194 